In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [48]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import argparse
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [49]:
dataset =pd.read_csv("../input/customer-personality-analysis/marketing_campaign.csv", sep="\t")
dataset.head()

In [50]:
dataset.isnull().sum()

In [51]:
median = dataset['Income'].median

In [52]:
median

In [53]:
# dataset['Income'].fillna(median, inplace = True)
# dataset['Income'].astype(int ,copy = True)
dataset = dataset.dropna()

In [54]:
# calculating the age of customers 
dataset['c_year'] = 2021
dataset['age'] = dataset['c_year']-dataset['Year_Birth']

# calculating the no of days as customers
dataset['c_date'] = '01-01-2015'
dataset['Dt_Customer'] =pd.to_datetime(dataset.Dt_Customer)
dataset['c_date'] = pd.to_datetime(dataset.c_date)
dataset['days_customer'] = (dataset['c_date']-dataset['Dt_Customer']).dt.days

# reducing some dimensions 

# defining two categories of expenses as on food ( fish+meat+ fruit) and as leisure_Expense ( wine, sweet, Gold)
dataset['leisure_Expense'] = dataset['MntWines']+ dataset['MntSweetProducts']+ dataset['MntGoldProds']
dataset['food'] = dataset['MntFishProducts'] + dataset['MntFruits'] + dataset['MntMeatProducts']

# defining accepted_any_cmp if customer have ever taken any campaign offer ( 1- yes, 0-no)
dataset['accepted_any_cmp']= dataset['AcceptedCmp1']+dataset['AcceptedCmp2']+dataset['AcceptedCmp3']+dataset['AcceptedCmp4']+dataset['AcceptedCmp5']+dataset['Response']
dataset['accepted_any_cmp']= np.where(dataset['accepted_any_cmp'] > 0, 1, 0)

# defining martial status as ( 0- single adult, 1- two adults)
mapping = {'Single' : 0, 'Together': 1, 'Married': 1, 'Divorced': 0, 'Widow': 0, 'Alone': 0,
       'Absurd': 0, 'YOLO': 0, 'Graduation': 1, 'PhD': 2, 'Master': 2, 'Basic': 0, '2n Cycle': 2}
dataset=dataset.replace({'Marital_Status': mapping, 'Education': mapping}) 

dataset['kid_teen'] = dataset['Kidhome'] + dataset['Teenhome']
dataset['kid_teen']= np.where(dataset['kid_teen'] > 0, 1, 0)

In [55]:
dataset=dataset[['Education', 'Marital_Status', 'Income', 'kid_teen',
       'Recency','leisure_Expense', 'food', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'accepted_any_cmp', 
       'Complain', 'age', 'days_customer']]

In [56]:
dataset.info()

In [57]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(dataset)

In [60]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,16):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,16), wcss)
plt.title('The elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')
plt.show()

In [61]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
scaled_data = pca.fit_transform(scaled_data)

In [65]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(scaled_data)

In [66]:
plt.scatter(scaled_data[y_kmeans ==0, 0], scaled_data[y_kmeans == 0, 1], s = 100, c = 'red', label='cluster1')
plt.scatter(scaled_data[y_kmeans ==1, 0], scaled_data[y_kmeans == 1, 1], s = 100, c = 'blue', label='cluster2')
plt.scatter(scaled_data[y_kmeans ==2, 0], scaled_data[y_kmeans == 2, 1], s = 100, c = 'green', label='cluster3')
plt.scatter(scaled_data[y_kmeans ==3, 0], scaled_data[y_kmeans == 3, 1], s = 100, c = 'cyan', label='cluster4')
plt.scatter(scaled_data[y_kmeans ==4, 0], scaled_data[y_kmeans == 4, 1], s = 100, c = 'gray', label='cluster5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s = 300, c = 'yellow', label = 'centroids')
plt.title('The clusters of Customers')
plt.xlabel('Component1')
plt.ylabel('Component2')
plt.legend()
plt.show()